In [1]:
!pip install pycld2
!pip install transformers
!apt-get install python-numpy libicu-dev
!pip install polyglot
!pip install PyICU
!pip install Morfessor
!pip install datasets

     |████████████████████████████████| 41.4MB 76kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833501 sha256=5acc3ac04dc6747676108fc815a4a52f3a5edee2adb47ace3da745767457ebeb
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2
     |████████████████████████████████| 1.3MB 15.3MB/s 
     |████████████████████████████████| 1.1MB 48.8MB/s 
     |████████████████████████████████| 2.9MB 46.9MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=09999f2726a8c29a77f3e662d22a0a7a2cca1756b9a317f173f9ab07ed641d42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-numpy is already the newe

# Importing all the libraries

In [2]:

from polyglot.downloader import downloader

import transformers
from transformers import BertForTokenClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm, trange
import numpy as np
import itertools
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from datasets import load_dataset
import pandas as pd
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



# The polyglot Collection

In [3]:
print(downloader.supported_languages_table("ner2", 3))

  1. Italian                    2. Hindi                      3. French                   
  4. Spanish; Castilian         5. Vietnamese                 6. Arabic                   
  7. Bulgarian                  8. Norwegian                  9. Estonian                 
 10. Japanese                  11. Greek, Modern             12. Slovene                  
 13. Korean                    14. Serbian                   15. Finnish                  
 16. Catalan; Valencian        17. Croatian                  18. Dutch                    
 19. Swedish                   20. Tagalog                   21. Danish                   
 22. Latvian                   23. Ukrainian                 24. Romanian, Moldavian, ... 
 25. Persian                   26. Slovak                    27. Portuguese               
 28. English                   29. Malay                     30. Polish                   
 31. German                    32. Indonesian                33. Chinese                  

# Downloading the data for German NER dataset

In [4]:
# downloading datasets
#data = load_dataset('polyglot_ner', 'de', split= 'train[:8000]')
train_dev_1k = load_dataset('polyglot_ner', 'de', split= 'train[1000:2000]')
train_dev_5k = load_dataset('polyglot_ner', 'de', split= 'train[3000:8000]')
test = load_dataset('polyglot_ner', 'de', split= 'train[9000:11000]')

Reusing dataset polyglot_ner (/root/.cache/huggingface/datasets/polyglot_ner/de/1.0.0/c929318589a30ee3f0dc5d53f1f99bf25a7ec16d3f319b3b671765c5ea464c99)
Reusing dataset polyglot_ner (/root/.cache/huggingface/datasets/polyglot_ner/de/1.0.0/c929318589a30ee3f0dc5d53f1f99bf25a7ec16d3f319b3b671765c5ea464c99)


Dataset polyglot_ner downloaded and prepared to /root/.cache/huggingface/datasets/polyglot_ner/de/1.0.0/c929318589a30ee3f0dc5d53f1f99bf25a7ec16d3f319b3b671765c5ea464c99. Subsequent calls will reuse this data.


In [5]:
train_dev_1k 

Dataset({
    features: ['id', 'lang', 'words', 'ner'],
    num_rows: 1000
})

In [6]:
train_dev_1k[3]

{'id': '1003',
 'lang': 'de',
 'ner': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'O',
  'LOC',
  'O',
  'O',
  'O',
  'O'],
 'words': ['Maria',
  'Grengg',
  '(*',
  '26',
  '.',
  'Februar',
  '1888',
  'in',
  'Stein',
  'an',
  'der',
  'Donau',
  ',',
  'Niederösterreich',
  ';',
  '†',
  '8',
  '.']}

# Convert the dataset into datafrane

In [7]:
df_train_1k = pd.DataFrame(train_dev_1k)
df_train_5k = pd.DataFrame(train_dev_5k)
df_test = pd.DataFrame(test) 

In [8]:
type(df_train_1k), type(df_train_5k)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [9]:
df_train_1k.shape, df_train_5k.shape, df_test.shape

((1000, 4), (5000, 4), (2000, 4))

In [10]:
df_train_1k.head(5)

,id,lang,ner,words
0,1000,de,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Inhalt, Die, Geschichte, spielt, auf, zwei, H..."
1,1001,de,"[O, O, O, O, O, O, O, O, O, O, O, O, O]","[Ab, 1940, war, sie, in, der, Dachorganisation..."
2,1002,de,"[O, O, O, ORG, O, O, O, O, O, O, O, O]","[Juni, 1980, in, Warschau, ), war, ein, polnis..."
3,1003,de,"[O, O, O, O, O, O, O, O, LOC, LOC, LOC, LOC, O...","[Maria, Grengg, (*, 26, ., Februar, 1888, in, ..."
4,1004,de,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Innerhalb, der, Partei, engagiert, sie, sich,..."


# Finding all the tags present in the data

In [11]:
tag_list_1k = list(set(list(itertools.chain.from_iterable(df_train_1k.ner.values))))
tag_list_5k = list(set(list(itertools.chain.from_iterable(df_train_5k.ner.values))))
tag_list_test = list(set(list(itertools.chain.from_iterable(df_test.ner.values))))

In [12]:
tag_list_1k, tag_list_5k, tag_list_test

(['ORG', 'O', 'LOC', 'PER'],
 ['ORG', 'O', 'LOC', 'PER'],
 ['ORG', 'O', 'LOC', 'PER'])

# Tag2Index generation

In [13]:
tag2index_1k = {t: i for i, t in enumerate(tag_list_1k)}
tag2index_5k = {t: i for i, t in enumerate(tag_list_5k)}
tag2index_test = {t: i for i, t in enumerate(tag_list_test)}

In [14]:
tag2index_1k, tag2index_5k, tag2index_test

({'LOC': 2, 'O': 1, 'ORG': 0, 'PER': 3},
 {'LOC': 2, 'O': 1, 'ORG': 0, 'PER': 3},
 {'LOC': 2, 'O': 1, 'ORG': 0, 'PER': 3})

# Using pretrained Bert tokenizer

In [15]:
# tokenizer = BertTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased', do_lower_case=False)

In [16]:
MAX = 512

# Encoding the dataset

In [17]:
encoded_dataset_1k = [tokenizer(" ".join(item['words']), return_tensors="pt", padding='max_length', truncation=True, max_length=MAX) for item in train_dev_1k]
encoded_dataset_5k = [tokenizer(" ".join(item['words']), return_tensors="pt", padding='max_length', truncation=True, max_length=MAX) for item in train_dev_5k]
encoded_dataset_test = [tokenizer(" ".join(item['words']), return_tensors="pt", padding='max_length', truncation=True, max_length=MAX) for item in test]

In [18]:
len(train_dev_1k[3]['words'])

18

In [19]:
" ".join(train_dev_1k[3]['words'])

'Maria Grengg (* 26 . Februar 1888 in Stein an der Donau , Niederösterreich ; † 8 .'

In [20]:
len([    3,  4281, 18707, 12436, 26954, 26978,  1636, 26914,  1466, 13705,
            50,  2513,   104,    21, 10444, 26918,  1559,  8744, 26968,     2,
           695, 26914,     4])

23

In [21]:
encoded_dataset_1k[3]

{'input_ids': tensor([[    3,  4281, 18707, 12436, 26954, 26978,  1636, 26914,  1466, 13705,
            50,  2513,   104,    21, 10444, 26918,  1559,  8744, 26968,     2,
           695, 26914,     4,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

# Adding label to the encoder

In [22]:
from keras.preprocessing.sequence import pad_sequences


In [23]:
# for lab in train_dev_1k['ner']:
#   print(lab)

In [24]:
train_dev_1k[0]['words']

['Inhalt',
 'Die',
 'Geschichte',
 'spielt',
 'auf',
 'zwei',
 'Handlungsebenen',
 ':',
 'In',
 'der',
 'Binnenerzählung',
 'flieht',
 'die',
 'junge',
 'Amelia',
 'Earnshawe',
 'vor',
 'Ghul',
 'en',
 'in',
 'eine',
 'alte',
 'Villa',
 '.']

In [25]:
tag2index_1k['PAD'] = 4
tag2index_5k['PAD'] = 4
tag2index_test['PAD'] = 4



In [26]:
train_dev_1k[0]['ner']

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [27]:
# tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in train_dev_1k['ner']],
#                      maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
#                      dtype="long", truncating="post")

In [28]:
# raise
import torch
for enc_item, item in zip(encoded_dataset_1k, train_dev_1k):
    list1 = list(tag2index_1k.get(t) for t in item['ner'])
    list1 += [tag2index_1k.get('PAD')] * (MAX - len(list1))
    enc_item['labels'] = torch.LongTensor(list1)

for enc_item, item in zip(encoded_dataset_5k, train_dev_5k):
    list1 = list(tag2index_5k.get(t) for t in item['ner'])
    list1 += [tag2index_5k.get('PAD')] * (MAX - len(list1))
    enc_item['labels'] = torch.LongTensor(list1)

for enc_item, item in zip(encoded_dataset_test, test):
    list1 = list(tag2index_test.get(t) for t in item['ner'])
    list1 += [tag2index_test.get('PAD')] * (MAX - len(list1))
    enc_item['labels'] = torch.LongTensor(list1)
    
    # enc_item['labels'] = torch.LongTensor(pad_sequences(list(tag2index.get(t) for t in item['ner']),maxlen=512,value = tag2index.get('O'),padding="post",
                      # dtype="long", truncating="post"))



In [29]:
encoded_dataset_1k[3]

{'input_ids': tensor([[    3,  4281, 18707, 12436, 26954, 26978,  1636, 26914,  1466, 13705,
            50,  2513,   104,    21, 10444, 26918,  1559,  8744, 26968,     2,
           695, 26914,     4,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [30]:
for key, val in encoded_dataset_1k[0].items():
    print(f'key: {key}, dimensions: {val.size()}')

key: input_ids, dimensions: torch.Size([1, 512])
key: token_type_ids, dimensions: torch.Size([1, 512])
key: attention_mask, dimensions: torch.Size([1, 512])
key: labels, dimensions: torch.Size([512])


In [31]:
for key, val in encoded_dataset_5k[0].items():
    print(f'key: {key}, dimensions: {val.size()}')

key: input_ids, dimensions: torch.Size([1, 512])
key: token_type_ids, dimensions: torch.Size([1, 512])
key: attention_mask, dimensions: torch.Size([1, 512])
key: labels, dimensions: torch.Size([512])


In [32]:
for key, val in encoded_dataset_test[0].items():
    print(f'key: {key}, dimensions: {val.size()}')

key: input_ids, dimensions: torch.Size([1, 512])
key: token_type_ids, dimensions: torch.Size([1, 512])
key: attention_mask, dimensions: torch.Size([1, 512])
key: labels, dimensions: torch.Size([512])


# Model declaration

In [33]:
model_1k = BertForTokenClassification.from_pretrained("bert-base-german-cased",num_labels=len(tag2index_1k))



Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [34]:
for item in encoded_dataset_1k:
    for key in item:
        item[key] = torch.squeeze(item[key])


In [35]:
for item in encoded_dataset_5k:
    for key in item:
        item[key] = torch.squeeze(item[key])


In [36]:
for item in encoded_dataset_test:
    for key in item:
        item[key] = torch.squeeze(item[key])


# Training for BERT fine tuning for 1000 datatset

In [37]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='results',
    logging_dir='logs',
    no_cuda=False,  # defaults to false anyway, just to be explicit
)

trainer = Trainer(
    model=model_1k,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=encoded_dataset_1k,
)

In [38]:
trainer.train()

Step,Training Loss
500,0.020711
1000,0.006624


TrainOutput(global_step=1250, training_loss=0.011571894836425782)

In [39]:
preds_1k = trainer.predict(encoded_dataset_test)

In [40]:
print(preds_1k.predictions[:2])
print(preds_1k.predictions[:2].argmax(-1))
print(preds_1k.label_ids[:2])
print(preds_1k.metrics)

[[[-3.1102946  8.459245  -3.3953965 -1.4527793 -2.9546711]
  [-2.760723   8.153156  -2.880376  -1.6139544 -3.3553832]
  [-2.9678957  7.8799953 -2.2806458 -2.115169  -3.3255794]
  ...
  [-2.4766183 -3.1633904 -2.6119568 -2.5619342 11.634041 ]
  [-2.4590707 -3.1244662 -2.550521  -2.5769382 11.625525 ]
  [-2.433801  -3.2108047 -2.5905154 -2.5853643 11.617461 ]]

 [[-3.1500163  7.818069  -1.9538072 -1.875944  -2.8171928]
  [-3.1875093  7.817202  -1.1090864 -1.6873327 -2.512389 ]
  [-2.542025   7.200176  -1.2936059 -1.8052053 -2.219724 ]
  ...
  [-2.4529865 -3.1718576 -2.616316  -2.5448608 11.638155 ]
  [-2.4364133 -3.1396112 -2.5556195 -2.555254  11.628906 ]
  [-2.4118042 -3.2068272 -2.6017113 -2.569868  11.624769 ]]]
[[1 1 1 ... 4 4 4]
 [1 1 1 ... 4 4 4]]
[[1 1 1 ... 4 4 4]
 [1 1 1 ... 4 4 4]]
{'eval_loss': 0.015569400042295456}


# Computing F1 score

In [41]:
import itertools
prediction_list_final = list(itertools.chain.from_iterable(preds_1k.label_ids))

In [42]:
joint_val = [val['labels'].numpy() for val in encoded_dataset_test]

In [43]:
preds_1k.label_ids[0].shape,joint_val[0].shape

((512,), (512,))

In [44]:
test_list_final = list(itertools.chain.from_iterable(joint_val))

In [45]:
len(test_list_final), len(prediction_list_final)

(1024000, 1024000)

In [46]:
test_list_final[-1],prediction_list_final[-1]

(4, 4)

In [47]:
type(test_list_final),type(prediction_list_final)

(list, list)

In [48]:
test_list_final = np.array(test_list_final)
prediction_list_final = np.array(prediction_list_final)

In [49]:
test_list_final = test_list_final[test_list_final != 4]
prediction_list_final = prediction_list_final[prediction_list_final != 4]

In [50]:
type(test_list_final),type(prediction_list_final)

(numpy.ndarray, numpy.ndarray)

In [51]:
preds_1k.label_ids.shape  # predicted values

(2000, 512)

In [52]:
len(encoded_dataset_test), encoded_dataset_test[0]['labels'].shape # true values

(2000, torch.Size([512]))

In [ ]:
# 
print("Validation/Testing F1-Micro Score: {}".format(f1_score(test_list_final, prediction_list_final, average='micro')))
print("Validation/Testing F1-Macro Score: {} \n".format(f1_score(test_list_final, prediction_list_final, average='macro')))

In [56]:
model_5k = BertForTokenClassification.from_pretrained("bert-base-german-cased",num_labels=len(tag2index_5k))

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

# Training for BERT fine tuning for 5000 datatset

In [57]:
training_args = TrainingArguments(
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='results',
    logging_dir='logs',
    no_cuda=False,  # defaults to false anyway, just to be explicit
)

trainer = Trainer(
    model=model_5k,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=encoded_dataset_5k,
)

In [58]:
trainer.train()

Step,Training Loss
500,0.021404
1000,0.012225
1500,0.009669
2000,0.008092
2500,0.007080


TrainOutput(global_step=2500, training_loss=0.01169389419555664)

In [ ]:
preds_5k = trainer.predict(test_set)

In [ ]:
print(preds_5k.predictions[:2])
print(preds_5k.predictions[:2].argmax(-1))
print(preds_5k.label_ids[:2])
print(preds_5k.metrics)

In [ ]:
prediction_list_final = list(itertools.chain.from_iterable(preds_5k.label_ids))

In [ ]:
joint_val = [val['labels'].numpy() for val in encoded_dataset_test]

In [ ]:
preds_5k.label_ids[0].shape,joint_val[0].shape

In [ ]:
test_list_final = list(itertools.chain.from_iterable(joint_val))

In [ ]:
len(test_list_final), len(prediction_list_final)

In [ ]:
test_list_final = np.array(test_list_final)
prediction_list_final = np.array(prediction_list_final)

In [ ]:
test_list_final = test_list_final[test_list_final != 4]
prediction_list_final = prediction_list_final[prediction_list_final != 4]

In [ ]:
type(test_list_final),type(prediction_list_final)

In [ ]:
preds_5k.label_ids.shape  # predicted values

In [ ]:
len(encoded_dataset_test), encoded_dataset_test[0]['labels'].shape # true values

In [85]:
print("Validation/Testing F1-Micro Score: {}".format(f1_score(test_list_final, prediction_list_final, average='micro')))
print("Validation/Testing F1-Macro Score: {} \n".format(f1_score(test_list_final, prediction_list_final, average='macro')))

Validation/Testing F1-Micro Score: 0.9426564872616838
Validation/Testing F1-Macro Score: 0.7466454326820232 



# Training for BERT fine tuning of 1000 datasets with frozen embeddings

In [41]:
model_1k = BertForTokenClassification.from_pretrained("bert-base-german-cased",num_labels=len(tag2index_1k))


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [42]:
for name, param in model_1k.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

In [47]:
training_args = TrainingArguments(
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='results',
    logging_dir='logs',
    no_cuda=False,  # defaults to false anyway, just to be explicit
)

trainer = Trainer(
    model=model_1k,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=encoded_dataset_1k,
)

In [48]:
trainer.train()

Step,Training Loss
500,0.146615


TrainOutput(global_step=500, training_loss=0.1466154327392578)

In [50]:
preds_1k_f = trainer.predict(encoded_dataset_test)

In [83]:
print("Validation/Testing F1-Micro Score: {}".format(f1_score(test_list_final, prediction_list_final, average='micro')))
print("Validation/Testing F1-Macro Score: {} \n".format(f1_score(test_list_final, prediction_list_final, average='macro')))

Validation/Testing F1-Micro Score: 0.93506411337945
Validation/Testing F1-Macro Score: 0.6824460919568094 



# Training for BERT fine tuning of 5000 datasets with frozen embeddings

In [51]:
model_5k = BertForTokenClassification.from_pretrained("bert-base-german-cased",num_labels=len(tag2index_5k))

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [52]:
for name, param in model_5k.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

In [53]:
training_args = TrainingArguments(
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='results',
    logging_dir='logs',
    no_cuda=False,  # defaults to false anyway, just to be explicit
)

trainer = Trainer(
    model=model_5k,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=encoded_dataset_5k,
)

In [54]:
trainer.train()

Step,Training Loss
500,0.140649
1000,0.048409
1500,0.044464
2000,0.042145
2500,0.040324


TrainOutput(global_step=2500, training_loss=0.06319811401367187)

In [55]:
preds_5k_f = trainer.predict(encoded_dataset_test)

In [87]:
print("Validation/Testing F1-Micro Score: {}".format(f1_score(test_list_final, prediction_list_final, average='micro')))
print("Validation/Testing F1-Macro Score: {} \n".format(f1_score(test_list_final, prediction_list_final, average='macro')))

Validation/Testing F1-Micro Score: 0.9386283111186098
Validation/Testing F1-Macro Score: 0.7348901984402518 

